In [1]:
from collections import defaultdict
from torchvision import transforms
from io import BytesIO
from tqdm import tqdm
from PIL import Image
import pandas as pd
import numpy as np
import torchvision
import requests
import pickle
import json
import time
import uuid
import re

## Скачка датасета GCC

In [11]:
def link_to_image(link): #функция для вывода фото в виде тензора, по ссылке данного фото
    try:
        response = requests.get(link, timeout=2)  #если ответа нет слишком долго, то фото пропускается
    except Exception:
        return None
    try:
        if response.status_code == 200:
            image = Image.open(BytesIO(response.content))
            transform = transforms.ToTensor()
            tensor_image = transform(image)
            return tensor_image
        else:
            return None
    except Exception:
        return None    
    
def image_to_tensor(tensor, filename): #сохранение фото в формате jpg с помощью тензора по определенному пути
    torchvision.utils.save_image(tensor, fp=filename)

In [8]:
data = pd.read_csv('Train_GCC-training.tsv', sep='\t', header=0, names=['text', 'link'])
data = data.sample(frac=1)

downloaded_count = 0

data_w_texts = pd.DataFrame(columns=['name', 'text']) 
for i, row in tqdm(data.iterrows(), total=data.shape[0]):
    if downloaded_count == 30000:
        break
    tensor = link_to_image(row['link'])
    try: 
        if tensor != None:
            filename = str(uuid.uuid4())
            image_to_tensor(tensor, filename=f"F:/Вера/Диплом/Датасеты/GCC/{filename}.jpeg")
            data_w_texts.loc[len(data_w_texts)] = [f"F:/Вера/Диплом/Датасеты/GCC/{filename}.jpeg", data['text'][i]]
            downloaded_count += 1
    except: 
        continue
        
data_w_texts.to_csv('F:/Вера/Диплом/Датасеты/GCC_all_data.csv')

  1%|▉                                                                   | 46278/3318332 [9:41:00<684:40:02,  1.33it/s]


UnicodeEncodeError: 'charmap' codec can't encode character '\xa3' in position 86: character maps to <undefined>

## Скачка датасета Flicker30k

In [29]:
annotations = defaultdict(list)
with open('results_20130124.token') as fh:
    for line in fh:
        img_id, caption = line.strip().split("\t")
        annotations[img_id[:-2]].append(caption)

In [58]:
def max_len_annotation(annotation):
    return sorted(annotation, key = lambda x: len(x))[-1]

'Two young guys with shaggy hair look at their hands while hanging out in the yard .'

In [60]:
flicker = pd.DataFrame(columns=['name', 'text']) 
for i in tqdm(range(len(annotations))):
    flicker.loc[len(flicker)] = [f"F:/Вера/Диплом/Датасеты/flickr30k-images/{list(annotations.keys())[i]}.jpeg", max_len_annotation(annotations[list(annotations.keys())[i]])]
flicker.to_csv('F:/Вера/Диплом/Датасеты/flicker_all_data.csv')

100%|████████████████████████████████████████████████████████████████████████████| 31783/31783 [05:35<00:00, 94.61it/s]


## Скачка датасета MS COCO

In [2]:
f = open('F:/Вера/Диплом/Датасеты/annotations_trainval2014/annotations/captions_val2014.json')
data = json.load(f)

images = data[list(data.keys())[1]]
anns = data[list(data.keys())[3]]

In [5]:
coco_set = pd.DataFrame(columns=['name', 'text']) 

for image in tqdm(images[:30000]):
    image_id = image['id']
    ann_list = []
    for ann in anns:
        ann_id = ann[list(anns[0].keys())[0]] 
        if ann_id == image_id:
            ann_list.append(ann[list(anns[0].keys())[2]])
    coco_set.loc[len(coco_set)] = [f"F:/Вера/Диплом/Датасеты/val2014/val2014/{image['file_name']}", sorted(ann_list, key = lambda x: len(x))[-1]]
coco_set.to_csv('F:/Вера/Диплом/Датасеты/coco_all_data.csv')

100%|██████████████████████████████████████████████████████████████████████████| 30000/30000 [1:17:00<00:00,  6.49it/s]


## Приведение датасетов к одному формату

In [ ]:
data = pd.read_csv('F:/Вера/Диплом/Датасеты/GCC_all_data.csv', index_col = 0)
data['text'] = data['text'].str.replace(';', '').values
data.columns = ['path', 'text']
data.to_csv('F:/Вера/Диплом/Датасеты/GCC_all_data.csv')

In [ ]:
data = pd.read_csv('F:/Вера/Диплом/Датасеты/flicker_all_data.csv', index_col = 0)
data.columns = ['path', 'text']
data['path'] = data['path'].str.replace('.jpeg', '').values
data = data.iloc[:30000]
data.to_csv('F:/Вера/Диплом/Датасеты/flicker_all_data.csv')

In [ ]:
data = pd.read_csv('F:/Вера/Диплом/Датасеты/coco_all_data.csv', index_col = 0)
data.columns = ['path', 'text']
data.to_csv('F:/Вера/Диплом/Датасеты/coco_all_data.csv')

## Исправление ошибок в датасете GCC

In [ ]:

new_data = pd.DataFrame(columns=['path', 'text'])
for i, row in data.iterrows():
    if row.isna().sum():
        new_data.loc[len(new_data)] = [re.search(r'\d+,(.+),"(.+)"', s).group(1), re.search(r'\d+,(.+),"(.+)"', s).group(2)]
    else:
        new_data.loc[len(new_data)] = row
        
new_data.to_csv('GCC_all_data.csv')

## Перевод описаний

In [ ]:
api_data = [
    {
        'api_key': 'AQVNzuPBvQ5y_nutJSYrlsTcBUCMnmQBkJn_Nmmg',
        'folder_id': 'b1g8gn09v76kh6c65l8u'
    },
    {
        'api_key': 'AQVN0DPxoZUjOr0cDkKDgyrGJx2CfaHG5-Z_dPIp',
        'folder_id': 'b1g7huvths893e6lgdn2'
    }
]

In [12]:
def translate_ru(sentence):

    api_key = 'AQVNzuPBvQ5y_nutJSYrlsTcBUCMnmQBkJn_Nmmg'
    folder_id = 'b1g8gn09v76kh6c65l8u'
    target_language = 'ru'
    
    body = {
        "targetLanguageCode": target_language,
        "texts": sentence,
        "folderId": folder_id,
    }

    headers = {
        "Content-Type": "application/json",
        "Authorization": "Api-Key {0}".format(api_key)
    }

    response = requests.post('https://translate.api.cloud.yandex.net/translate/v2/translate',
        json = body,
        headers = headers
    )

    return response.json()['translations'][0]['text']

In [3]:
data = pd.read_csv('F:/Вера/Диплом/Датасеты/GCC_all_data.csv', index_col = 0)
translations = []
for text in tqdm(data['text'].values):
    try:
        t = translate_ru(text)
    except:
        t = translate_ru(text)
    translations.append(t)
data['translations'] = translations

100%|██████████████████████████████████████████████████████████████████████████| 30000/30000 [2:19:18<00:00,  3.59it/s]


In [8]:
data.to_csv('F:/Вера/Диплом/Датасеты/GCC_all_data.csv')

In [8]:
data = pd.read_csv('F:/Вера/Диплом/Датасеты/flicker_all_data.csv', index_col = 0)
translations = []
for text in tqdm(data['text'].values):
    try:
        t = translate_ru(text)
    except:
        t = translate_ru(text)
    translations.append(t)
data['translations'] = translations

100%|██████████████████████████████████████████████████████████████████████████| 30000/30000 [2:53:28<00:00,  2.88it/s]


In [10]:
data.to_csv('F:/Вера/Диплом/Датасеты/flicker_all_data.csv')

In [13]:
data = pd.read_csv('F:/Вера/Диплом/Датасеты/coco_all_data.csv', index_col = 0)
translations = []
for text in tqdm(data['text'].values):
    try:
        t = translate_ru(text)
    except:
        t = translate_ru(text)
    translations.append(t)
data['translations'] = translations

100%|██████████████████████████████████████████████████████████████████████████| 30000/30000 [2:37:42<00:00,  3.17it/s]


In [15]:
data.to_csv('F:/Вера/Диплом/Датасеты/coco_all_data.csv')

## Запись картинок в виде тензоров

In [3]:
def path_tensor(path):
    image = Image.open(path)
    tensor = transform(image)
    return tensor

In [2]:
transform = transforms.Compose([
    transforms.PILToTensor(),
    transforms.Resize(size=(224,224))
])

In [4]:
tqdm.pandas()
data = pd.read_csv('F:/Вера/Диплом/Датасеты/GCC_all_data.csv', index_col=0)
data['tensor'] = data['path'].progress_apply(lambda x: path_tensor(x))

100%|████████████████████████████████████████████████████████████████████████████| 30000/30000 [12:13<00:00, 40.89it/s]


,path,text,translations,tensor
0,F:/Вера/Диплом/Датасеты/GCC/7b82b7ca-ce2f-43d4...,the little knownqualified rugby players playin...,"малоизвестные квалифицированные регбисты, игра...","[[[tensor(240, dtype=torch.uint8), tensor(213,..."
1,F:/Вера/Диплом/Датасеты/GCC/b3f5adbb-1492-4719...,a red checkered vector background with grunge ...,векторный фон в красную клетку с гранжевой тек...,"[[[tensor(238, dtype=torch.uint8), tensor(239,..."
2,F:/Вера/Диплом/Датасеты/GCC/2bbcd6d9-8ada-4cad...,man and woman sitting under umbrella with pood...,мужчина и женщина сидят под зонтиком с пуделем...,"[[[tensor(158, dtype=torch.uint8), tensor(158,..."
3,F:/Вера/Диплом/Датасеты/GCC/1d09d550-c1a6-47b1...,makes an excellent shield from the chill .,получается отличная защита от холода.,"[[[tensor(70, dtype=torch.uint8), tensor(69, d..."
4,F:/Вера/Диплом/Датасеты/GCC/570a145c-3b86-4216...,a wild horse makes its way across snow covered...,дикая лошадь пробирается по заснеженным полям ...,"[[[tensor(48, dtype=torch.uint8), tensor(53, d..."
...,...,...,...,...
29995,F:/Вера/Диплом/Датасеты/GCC/b89ea829-32f0-4845...,find out the best bangs for your face shape !,"узнайте, какая челка лучше всего подходит для ...","[[[tensor(250, dtype=torch.uint8), tensor(250,..."
29996,F:/Вера/Диплом/Датасеты/GCC/55b61296-3732-40f3...,vector illustration of stylish guys on a dirty...,векторная иллюстрация стильных парней на грязн...,"[[[tensor(255, dtype=torch.uint8), tensor(255,..."
29997,F:/Вера/Диплом/Датасеты/GCC/52aa43f9-a0bc-4d9b...,the back porch of person and home .,заднее крыльцо человека и дома .,"[[[tensor(191, dtype=torch.uint8), tensor(220,..."
29998,F:/Вера/Диплом/Датасеты/GCC/f52d19e0-4cdc-418a...,at times your hot - air balloon will glide jus...,время от времени ваш воздушный шар будет сколь...,"[[[tensor(254, dtype=torch.uint8), tensor(252,..."


In [7]:
with open('F:/Вера/Диплом/Датасеты/GCC_data.pickle', 'wb') as f:
    pickle.dump(data, f)

ValueError: I/O operation on closed file.

In [7]:
tqdm.pandas()
data = pd.read_csv('F:/Вера/Диплом/Датасеты/coco_all_data.csv', index_col=0)
data['tensor'] = data['path'].progress_apply(lambda x: path_tensor(x))

100%|████████████████████████████████████████████████████████████████████████████| 30000/30000 [17:18<00:00, 28.90it/s]


,path,text,translations,tensor
0,F:/Вера/Диплом/Датасеты/val2014/val2014/COCO_v...,A dirt path with a young person on a motor bik...,Грунтовая дорожка с молодым человеком на мотоц...,"[[[tensor(255, dtype=torch.uint8), tensor(255,..."
1,F:/Вера/Диплом/Датасеты/val2014/val2014/COCO_v...,A woman marking a cake with the back of a chef...,Женщина делает пометку на торте тыльной сторон...,"[[[tensor(159, dtype=torch.uint8), tensor(159,..."
2,F:/Вера/Диплом/Датасеты/val2014/val2014/COCO_v...,a young boy barefoot holding an umbrella touch...,"маленький мальчик босиком держит зонтик, касая...","[[[tensor(237, dtype=torch.uint8), tensor(242,..."
3,F:/Вера/Диплом/Датасеты/val2014/val2014/COCO_v...,a little boy wearing headphones and looking at...,маленький мальчик в наушниках и смотрит на мон...,"[[[tensor(98, dtype=torch.uint8), tensor(82, d..."
4,F:/Вера/Диплом/Датасеты/val2014/val2014/COCO_v...,a boy wearing headphones using one computer in...,"мальчик в наушниках, пользующийся одним компью...","[[[tensor(132, dtype=torch.uint8), tensor(179,..."
...,...,...,...,...
29995,F:/Вера/Диплом/Датасеты/val2014/val2014/COCO_v...,"Two giraffes behind a wall of boulders, in fro...","Два жирафа за стеной из валунов, перед воротами.","[[[tensor(4, dtype=torch.uint8), tensor(4, dty..."
29996,F:/Вера/Диплом/Датасеты/val2014/val2014/COCO_v...,A bunch of cars driving down a street surround...,"Группа машин едет по улице, окруженной высоким...","[[[tensor(147, dtype=torch.uint8), tensor(150,..."
29997,F:/Вера/Диплом/Датасеты/val2014/val2014/COCO_v...,A lone giraffe standing under trees near an op...,"Одинокий жираф, стоящий под деревьями возле ав...","[[[tensor(109, dtype=torch.uint8), tensor(108,..."
29998,F:/Вера/Диплом/Датасеты/val2014/val2014/COCO_v...,A sign that might read parking here where bicy...,"Знак, который мог бы гласить: ""Парковка здесь,...","[[[tensor(193, dtype=torch.uint8), tensor(192,..."


In [8]:
with open('F:/Вера/Диплом/Датасеты/coco_data.pickle', 'wb') as f:
    pickle.dump(data, f)

In [9]:
tqdm.pandas()
data = pd.read_csv('F:/Вера/Диплом/Датасеты/flicker_all_data.csv', index_col=0)
data['tensor'] = data['path'].progress_apply(lambda x: path_tensor(x))

100%|████████████████████████████████████████████████████████████████████████████| 30000/30000 [14:52<00:00, 33.61it/s]


,path,text,translations,tensor
0,F:/Вера/Диплом/Датасеты/flickr30k-images/10000...,Two young guys with shaggy hair look at their ...,Два молодых парня с лохматыми волосами смотрят...,"[[[tensor(3, dtype=torch.uint8), tensor(4, dty..."
1,F:/Вера/Диплом/Датасеты/flickr30k-images/10002...,Several men in hard hats are operating a giant...,Несколько мужчин в касках управляют гигантской...,"[[[tensor(154, dtype=torch.uint8), tensor(157,..."
2,F:/Вера/Диплом/Датасеты/flickr30k-images/10002...,A child in a pink dress is climbing up a set o...,Ребенок в розовом платье поднимается по лестни...,"[[[tensor(76, dtype=torch.uint8), tensor(106, ..."
3,F:/Вера/Диплом/Датасеты/flickr30k-images/10003...,Someone in a blue shirt and hat is standing on...,Кто-то в синей рубашке и шляпе стоит на лестни...,"[[[tensor(110, dtype=torch.uint8), tensor(110,..."
4,F:/Вера/Диплом/Датасеты/flickr30k-images/10003...,"Two men , one in a gray shirt , one in a black...","Двое мужчин , один в серой рубашке , другой в ...","[[[tensor(115, dtype=torch.uint8), tensor(113,..."
...,...,...,...,...
29995,F:/Вера/Диплом/Датасеты/flickr30k-images/72491...,"A group of men wearing black suits , yellow sw...","Группа мужчин в черных костюмах, желтых плават...","[[[tensor(46, dtype=torch.uint8), tensor(170, ..."
29996,F:/Вера/Диплом/Датасеты/flickr30k-images/72491...,Many sports people dressed in black swim wear ...,"Многие спортивные люди, одетые в черные плавки...","[[[tensor(7, dtype=torch.uint8), tensor(12, dt..."
29997,F:/Вера/Диплом/Датасеты/flickr30k-images/72497...,A group of people surrounding a man in a black...,"Группа людей, окруживших мужчину в черно-красн...","[[[tensor(137, dtype=torch.uint8), tensor(144,..."
29998,F:/Вера/Диплом/Датасеты/flickr30k-images/72521...,A woman in a brown coat and black shirt smiles...,Женщина в коричневом пальто и черной рубашке у...,"[[[tensor(16, dtype=torch.uint8), tensor(17, d..."


In [10]:
with open('F:/Вера/Диплом/Датасеты/flicker_data.pickle', 'wb') as f:
    pickle.dump(data, f)

## Создание тестового набора данных

In [6]:
with open('F:/Вера/Диплом/Датасеты/GCC_data.pickle', 'rb') as f:
    data = pickle.load(f)
    
train_data = data.iloc[:24000]
test_data = data.iloc[24000:]

with open('F:/Вера/Диплом/Датасеты/GCC_train.pickle', 'wb') as f:
    pickle.dump(train_data, f)
    
with open('F:/Вера/Диплом/Датасеты/all_test.pickle', 'wb') as f:
    pickle.dump(test_data, f)

In [11]:
with open('F:/Вера/Диплом/Датасеты/coco_data.pickle', 'rb') as f:
    data = pickle.load(f)
    
train_data = data.iloc[:24000]
test_data = data.iloc[24000:]

with open('F:/Вера/Диплом/Датасеты/all_test.pickle', 'rb') as f:
    test_dataset = pickle.load(f)
    
test_concat = pd.concat([test_data, test_dataset]).reset_index(drop=True)

with open('F:/Вера/Диплом/Датасеты/coco_train.pickle', 'wb') as f:
    pickle.dump(train_data, f)
    
with open('F:/Вера/Диплом/Датасеты/all_test.pickle', 'wb') as f:
    pickle.dump(test_concat, f)

In [2]:
with open('F:/Вера/Диплом/Датасеты/flicker_data.pickle', 'rb') as f:
    data = pickle.load(f)
    
train_data = data.iloc[:24000]
test_data = data.iloc[24000:]

with open('F:/Вера/Диплом/Датасеты/all_test.pickle', 'rb') as f:
    test_dataset = pickle.load(f)
    
test_concat = pd.concat([test_data, test_dataset]).reset_index(drop=True)

with open('F:/Вера/Диплом/Датасеты/flicker_train.pickle', 'wb') as f:
    pickle.dump(train_data, f)
    
with open('F:/Вера/Диплом/Датасеты/all_test.pickle', 'wb') as f:
    pickle.dump(test_concat, f)